Importamos librerias

In [56]:
import numpy as np
import pandas as pd


In [57]:
import sys
try:
    import cplex
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install cplex
    else:
        !pip install --user cplex

In [58]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

In [59]:
import docplex.cp
from docplex.cp.model import *


Visualizamos los dataframes con los parametros y los productos

In [60]:
dfproductos = pd.read_csv("productos.csv")

In [61]:
dfproductos

,Unnamed: 0,nombreprod,idprod,stockInicial,demandaMensual,precioUSD,pedidoMinimo,provedorID
0,0,prod1,1.0,300.0,33.0,243.0,158.0,15.0
1,0,prod2,2.0,253.0,8.0,805.0,71.0,5.0
2,0,prod3,3.0,69.0,86.0,612.0,222.0,5.0
3,0,prod4,4.0,261.0,42.0,378.0,201.0,4.0
4,0,prod5,5.0,320.0,74.0,259.0,164.0,6.0
5,0,prod6,6.0,15.0,79.0,250.0,180.0,3.0
6,0,prod7,7.0,375.0,34.0,368.0,154.0,12.0
7,0,prod8,8.0,30.0,117.0,380.0,237.0,12.0
8,0,prod9,9.0,172.0,69.0,532.0,91.0,1.0
9,0,prod10,10.0,93.0,59.0,630.0,110.0,15.0


In [62]:
dfproductos["demandaMensual"][1]

8.0

In [63]:
dfProvedores = pd.read_csv("provedores.csv")

In [64]:
dfProvedores

,Unnamed: 0,provedor,provedorID,minimoMontoPedido,puertoEnvio
0,0,prov1,1.0,7192.0,2.0
1,0,prov2,2.0,13347.0,3.0
2,0,prov3,3.0,9037.0,1.0
3,0,prov4,4.0,7526.0,5.0
4,0,prov5,5.0,9666.0,3.0
5,0,prov6,6.0,10737.0,5.0
6,0,prov7,7.0,14051.0,1.0
7,0,prov8,8.0,11337.0,4.0
8,0,prov9,9.0,10462.0,3.0
9,0,prov10,10.0,8407.0,5.0


In [65]:
dfGeneral = pd.read_csv("parametrosGenerales.csv")

In [66]:
dfGeneral

,Unnamed: 0,interesPlataMuerta,meses,productos,provedores,puertos,costoDespachante,delayEntrega
0,0,0.025,24,500,15,5,600,4


In [67]:
dfGeneral["interesPlataMuerta"][0]

0.025

Nos creamos el conjunto para iterar de cada tipo

In [68]:
Meses = list(range(1,int(dfGeneral["meses"] + 1)))
Prods = list(range(1,int(dfGeneral["productos"] + 1)))
Provs = list(range(1,int(dfGeneral["provedores"] + 1)))
Puertos = list(range(1,int(dfGeneral["puertos"] + 1)))

In [69]:
Meses

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

Definimos el modelo

In [70]:
mdl = CpoModel(name="modeladoImportacion")

Definimos las variables de costos acumuladas

In [71]:
varCostosAcumulados = {}
varAduanas = {}
varPenalizacionPlataMuerta = {}
for m in Meses:
    varCostosAcumulados[m] = integer_var(name="costosAcum_m".format(m))
    varAduanas[m] = integer_var(name="aduanas_m".format(m))
    varPenalizacionPlataMuerta[m] = integer_var(name="penalPlata_m".format(m))

In [72]:
mdl.add(varCostosAcumulados[1] == 0) #  El costo inicial el primer mes es 0


Definimos la funcion objetivo como la minimizacion de los costos acumulados hasta el ultimo mes

In [73]:
mdl.add(minimize(varCostosAcumulados[len(Meses)]+varPenalizacionPlataMuerta[len(Meses)]*dfGeneral["interesPlataMuerta"][0]))

Definicion a traves de restriccion de que es el costo acumulado

In [74]:
for m in Meses:
    if(m>=2):
        mdl.add(
            varCostosAcumulados[m] >= 
                varCostosAcumulados[m-1] + 
                varAduanas[m-1]*dfGeneral["costoDespachante"][0] +
                varPenalizacionPlataMuerta[m-1]*dfGeneral["interesPlataMuerta"][0]) 


In [75]:
varStockProdMes = {}
for m in Meses:
    for i in Prods:
        varStockProdMes[(i, m)] = integer_var(name="stock_produc_mes".format(i, m))

In [76]:
for m in Meses:
    suma = 0
    for i in Prods:
        suma = suma + (varStockProdMes[(i, m)]- dfproductos["demandaMensual"][i-1]) * dfproductos["precioUSD"][i-1]
    mdl.add(
        varPenalizacionPlataMuerta[m] >= suma
               )

In [77]:
delay = dfGeneral["delayEntrega"][0]

for m in range(delay+1, Meses[len(Meses)-1] + 1):
    for i in Prods:
       mdl.add(varStockProdMes[(i, m)] >= dfproductos["demandaMensual"][i-1])

In [78]:
deltaProdProvMes = {}
cantProdProvMes = {}

for m in Meses:
    for i in Prods:
        deltaProdProvMes[(i, m)] = binary_var(name="delta_prov_produc_mes".format(i, m))
        cantProdProvMes[(i, m)] = integer_var(name="cant_prov_produc_mes".format(i, m))

In [79]:
thetaProvMes = {}

for m in Meses:
        for p in Provs:
            thetaProvMes[( p,m)] = binary_var(name="theta_prov__mes".format( p, m))

In [80]:
for m in Meses:
    for i in Prods:
            mdl.add(deltaProdProvMes[(i,m)] * dfproductos["pedidoMinimo"][i-1] <= cantProdProvMes[(i,m)])

In [81]:
M = 1000000 # NUNCA SE PIDE MAS DE 1 MILLON DE UN PRODUCTO, y si es que pedis la indicadora es 1
for m in Meses:
    for i in Prods:
            mdl.add(deltaProdProvMes[(i,m)] * M >= cantProdProvMes[(i,m)])

In [82]:
for m in Meses:
    for p in Provs:
        sumaIndicadoras = 0
        for i in Prods:
            if(dfproductos["provedorID"][i-1] == p):
                sumaIndicadoras = sumaIndicadoras + deltaProdProvMes[(i,m)]
        mdl.add(sumaIndicadoras <= M *  thetaProvMes[( p,m)])

In [83]:
for m in Meses:
    for p in Provs:
        sumaIndicadoras2 = 0
        for i in Prods:
            if(dfproductos["provedorID"][i-1] == p):
                sumaIndicadoras2 = sumaIndicadoras2 + deltaProdProvMes[(i, m)]
        mdl.add(sumaIndicadoras2 >=  thetaProvMes[( p,m)])

In [84]:
for m in Meses:
    for p in Provs:
        sumaAcumulado = 0
        for i in Prods:
            if(dfproductos["provedorID"][i-1] == p):
                sumaAcumulado = sumaAcumulado + cantProdProvMes[(i,m)]*dfproductos["precioUSD"][i-1]
        mdl.add(sumaAcumulado >= dfProvedores["minimoMontoPedido"][p-1] *  thetaProvMes[( p,m)])

In [85]:
for i in Prods:
    mdl.add(varStockProdMes[(i, 1)] == dfproductos["stockInicial"][i-1])

In [86]:
delay = dfGeneral["delayEntrega"][0]
for m in range(delay+1, Meses[len(Meses)-1] + 1):
    for i in Prods:
        sumaCantidad = 0
        mdl.add(varStockProdMes[(i, m)] == varStockProdMes[(i, m-1)] - 
                                            dfproductos["demandaMensual"][i-1] + 
                                               cantProdProvMes[(i,m - delay)] )

In [87]:
for m in range(2, delay+1):
    for i in Prods:
        if(dfproductos["stockInicial"][i-1] - dfproductos["demandaMensual"][i-1]*(m-1)>=0):
            mdl.add(varStockProdMes[(i, m)] == varStockProdMes[(i, m-1)] - dfproductos["demandaMensual"][i-1])
        else:
            mdl.add(varStockProdMes[(i, m)] == 0)

In [88]:
puertPuertoMes = {}

for m in Meses:
        for k in Puertos:
            puertPuertoMes[( k,m)] = binary_var(name="puerto__mes".format( k, m))

In [89]:
for m in Meses:
    for k in Puertos:
        for p in Provs:
            if(dfProvedores["puertoEnvio"][p-1] == k):
                mdl.add(puertPuertoMes[( k,m)] >= thetaProvMes[( p,m)])

In [90]:
for m in Meses:
    sumaPuertos = 0
    for k in Puertos:
        sumaPuertos = sumaPuertos + puertPuertoMes[( k,m)]
    mdl.add(varAduanas[m] == sumaPuertos)#-9007199254740991) # ACA DEBERIA IR SUMA PUERTOS

In [92]:
msol = mdl.solve(TimeLimit=1800) #3600 es 1hra de espera

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 36552 variables, 47512 constraints
 ! Presolve      : 1081 extractables eliminated
 ! TimeLimit            = 1800
 ! Initial process time : 1.49s (1.49s extraction + 0.01s propagation)
 !  . Log search space  : 473734.9 (before), 473734.9 (after)
 !  . Memory usage      : 31.9 MB (before), 31.9 MB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0      36552                 -
 + New bound is 3.799271e+06
                        0      33688    1   F        -
 + New bound is 3.804647e+06
                     1000      28281    1         0  = delta_prov_produc_mes@6141
                     1000      27697    2         0  = delta_prov_produc_mes@4542
                     1000      27518    3         0  = delta_prov_prod

    7.633596e+07    10000      11899    3         0  = delta_prov_produc_mes@5994
    7.633596e+07    26000      23537    4     708618  = cant_prov_produc_mes@11841
    7.633596e+07    25000      26856    1         0  = delta_prov_produc_mes@8923
    7.633596e+07    26000      27122    2     4206178  = stock_produc_mes@4428
    7.633596e+07    27000      21079    4     5049316  = stock_produc_mes@7218
    7.633596e+07    26000      24450    1         0  = delta_prov_produc_mes@6951
    7.633596e+07    27000      24689    2     4605511  = stock_produc_mes@7932
    7.633596e+07    28000      18555    4     559798  = cant_prov_produc_mes@1459
    7.633596e+07    27000      22097    1     440716  = cant_prov_produc_mes@2840
    7.633596e+07    28000      22188    2     94074  = cant_prov_produc_mes@9183
    7.633596e+07    11000       4364    3        71  = stock_produc_mes@11826
 *  7.628545e+07    11274  220.50s      3      (gap is 95.01%)
    7.628545e+07    29000      15802    4       

    7.628545e+07    25000      16057    3         0  = delta_prov_produc_mes@8640
    7.628545e+07    49000      10404    4         0  = delta_prov_produc_mes@7482
 ! Time = 416.39s, Average fail depth = 528, Memory usage = 972.1 MB
 ! Current bound is 3.804647e+06 (gap is 95.01%)
 !          Best Branches  Non-fixed    W       Branch decision
    7.628545e+07    52000          1    1         0  = cant_prov_produc_mes@1445
    7.628545e+07    52000       1666    2     997039  = cant_prov_produc_mes@3505
    7.628545e+07    50000          1    4     19999280  = stock_produc_mes@11772
    7.628545e+07    53000      28014    1         0  = delta_prov_produc_mes@7304
    7.628545e+07    53000        102    2     939254  = cant_prov_produc_mes@11124
    7.628545e+07    51000       2690    4     1000000  = cant_prov_produc_mes@11282
    7.628545e+07    54000          1    2         0  = delta_prov_produc_mes@5785
    7.628545e+07    52000        690    4     1000000  = cant_prov_produc_mes@1

    7.451867e+07    53000      29037    3         0  = delta_prov_produc_mes@2431
    7.451867e+07    76000      21200    4         0  = delta_prov_produc_mes@564
    7.451867e+07    77000      18765    4     1035507  = stock_produc_mes@7031
    7.451867e+07    75000      12669    2     936438  = cant_prov_produc_mes@3547
    7.451867e+07    54000      26858    3         0  = delta_prov_produc_mes@3485
    7.451867e+07    78000      16160    4     123013  = cant_prov_produc_mes@1582
    7.451867e+07    55000      24712    3         0  = delta_prov_produc_mes@4497
    7.451867e+07    79000      13475    4      2438  = cant_prov_produc_mes@1918
    7.451867e+07    76000      10081    2     3287313  = stock_produc_mes@8928
    7.451867e+07    80000      10844    4     548120  = stock_produc_mes@2203
 ! Time = 827.08s, Average fail depth = 5517, Memory usage = 1.5 GB
 ! Current bound is 3.804647e+06 (gap is 94.89%)
 !          Best Branches  Non-fixed    W       Branch decision
    7.45186

 *  7.450941e+07    92740  1439.60s     3      (gap is 94.89%)
    7.450941e+07    93000          1    3     519757  = stock_produc_mes@7900
    7.450941e+07    94000      31814    3         0  = delta_prov_produc_mes@1062
    7.450941e+07    95000      29431    3         0  = delta_prov_produc_mes@2251
    7.450941e+07    96000      27238    3         0  = delta_prov_produc_mes@3302
    7.450941e+07    65000      13708    1         0  = delta_prov_produc_mes@4415
    7.450941e+07    97000      25097    3         0  = delta_prov_produc_mes@4319
    7.450941e+07    98000      22967    3         0  = delta_prov_produc_mes@5326
    7.450941e+07    87000      20488    4         0  = delta_prov_produc_mes@11002
    7.450941e+07    99000      20857    3         0  = delta_prov_produc_mes@6334
    7.450941e+07     111k          1    2         0  = cant_prov_produc_mes@6541
    7.450941e+07    66000      10062    1         0  = delta_prov_produc_mes@3506
    7.450941e+07    88000      15675   

In [94]:
msol.get_value(varCostosAcumulados[12])

43541627

In [95]:
msol.get_value(varStockProdMes[(1,8)])

69

In [96]:
msol.get_value(varPenalizacionPlataMuerta[1])

35341836

In [ ]:
msol.get_value(varPenalizacionPlataMuerta[8])

In [ ]:
msol.get_value(puertPuertoMes[1,8])

In [98]:
stock2 = []
for m in Meses:
    stock2.append(msol.get_value(varStockProdMes[(1,m)]))

In [99]:
stock2

[300,
 267,
 234,
 201,
 168,
 135,
 102,
 69,
 36,
 495,
 462,
 429,
 396,
 363,
 330,
 297,
 264,
 231,
 198,
 165,
 132,
 99,
 66,
 33]

In [100]:
cantPedidos = 0
for m in Meses:
    for p in Provs:
        if(msol.get_value(thetaProvMes[( p,m)])==1):
            print("en mes "+ str(m) + "al prov " + str(p))
        cantPedidos = cantPedidos + msol.get_value(thetaProvMes[( p,m)])

en mes 1al prov 1
en mes 1al prov 2
en mes 1al prov 3
en mes 1al prov 4
en mes 1al prov 5
en mes 1al prov 6
en mes 1al prov 7
en mes 1al prov 8
en mes 1al prov 9
en mes 1al prov 10
en mes 1al prov 11
en mes 1al prov 12
en mes 1al prov 13
en mes 1al prov 14
en mes 1al prov 15
en mes 2al prov 1
en mes 2al prov 2
en mes 2al prov 3
en mes 2al prov 4
en mes 2al prov 5
en mes 2al prov 6
en mes 2al prov 7
en mes 2al prov 9
en mes 2al prov 10
en mes 2al prov 11
en mes 2al prov 12
en mes 2al prov 13
en mes 2al prov 14
en mes 2al prov 15
en mes 3al prov 2
en mes 3al prov 4
en mes 3al prov 6
en mes 3al prov 7
en mes 3al prov 8
en mes 3al prov 9
en mes 3al prov 10
en mes 4al prov 1
en mes 4al prov 2
en mes 4al prov 3
en mes 4al prov 4
en mes 4al prov 7
en mes 4al prov 12
en mes 4al prov 13
en mes 4al prov 14
en mes 4al prov 15
en mes 5al prov 4
en mes 5al prov 6
en mes 5al prov 13
en mes 5al prov 15
en mes 6al prov 3
en mes 6al prov 6
en mes 6al prov 9
en mes 6al prov 10
en mes 6al prov 11
en mes 

In [101]:
for m in Meses:
    print()
    print("En el mes " + str(m))
    for p in Provs:
        if(msol.get_value(thetaProvMes[( p,m)])==1):
            print()
            print("Se hizo un pedido al provedor " + str(p) + " donde se pidio:")
            for i in Prods:
                if(dfproductos["provedorID"][i-1] == p and msol.get_value(cantProdProvMes[(i,m)])>0 ):
                    print("Del producto " + str(i) + "una cantidad de " + str(msol.get_value(cantProdProvMes[(i,m)])))


En el mes 1

Se hizo un pedido al provedor 1 donde se pidio:
Del producto 9una cantidad de 1449
Del producto 26una cantidad de 1738
Del producto 39una cantidad de 1554
Del producto 71una cantidad de 770
Del producto 76una cantidad de 2100
Del producto 86una cantidad de 2100
Del producto 92una cantidad de 1491
Del producto 123una cantidad de 546
Del producto 197una cantidad de 1365
Del producto 205una cantidad de 1236
Del producto 216una cantidad de 1968
Del producto 258una cantidad de 1625
Del producto 263una cantidad de 1825
Del producto 315una cantidad de 1113
Del producto 328una cantidad de 2436
Del producto 363una cantidad de 1134
Del producto 378una cantidad de 1410
Del producto 383una cantidad de 1643
Del producto 418una cantidad de 1739
Del producto 458una cantidad de 1953
Del producto 460una cantidad de 1806
Del producto 490una cantidad de 1827

Se hizo un pedido al provedor 2 donde se pidio:
Del producto 18una cantidad de 630
Del producto 20una cantidad de 1995
Del producto 5

Del producto 203una cantidad de 352
Del producto 266una cantidad de 799
Del producto 407una cantidad de 1084

Se hizo un pedido al provedor 2 donde se pidio:
Del producto 268una cantidad de 1288
Del producto 435una cantidad de 1392
Del producto 442una cantidad de 1431

Se hizo un pedido al provedor 3 donde se pidio:
Del producto 48una cantidad de 1214
Del producto 362una cantidad de 625
Del producto 425una cantidad de 987

Se hizo un pedido al provedor 4 donde se pidio:
Del producto 34una cantidad de 673
Del producto 156una cantidad de 1163
Del producto 213una cantidad de 814

Se hizo un pedido al provedor 5 donde se pidio:
Del producto 283una cantidad de 835
Del producto 330una cantidad de 1087

Se hizo un pedido al provedor 6 donde se pidio:
Del producto 220una cantidad de 1201
Del producto 243una cantidad de 1120
Del producto 338una cantidad de 428

Se hizo un pedido al provedor 7 donde se pidio:
Del producto 30una cantidad de 433
Del producto 257una cantidad de 954
Del producto 452

Del producto 32una cantidad de 92
Del producto 239una cantidad de 205

Se hizo un pedido al provedor 12 donde se pidio:
Del producto 474una cantidad de 147

Se hizo un pedido al provedor 13 donde se pidio:
Del producto 450una cantidad de 131

En el mes 18

Se hizo un pedido al provedor 2 donde se pidio:
Del producto 493una cantidad de 196

Se hizo un pedido al provedor 7 donde se pidio:
Del producto 364una cantidad de 64

En el mes 19

Se hizo un pedido al provedor 4 donde se pidio:
Del producto 426una cantidad de 81

Se hizo un pedido al provedor 13 donde se pidio:
Del producto 196una cantidad de 162

En el mes 20

Se hizo un pedido al provedor 4 donde se pidio:
Del producto 498una cantidad de 193

Se hizo un pedido al provedor 8 donde se pidio:
Del producto 500una cantidad de 241

Se hizo un pedido al provedor 9 donde se pidio:
Del producto 295una cantidad de 59

En el mes 21

En el mes 22

En el mes 23

En el mes 24
